In [ ]:
import os
import random
from typing import Dict, List, Optional, Tuple, Union
import openai
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Setup OpenAI client
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


class VietnameseUserSimulator:
    def __init__(self, persona_type: str = "random"):
        """
        Initialize the Vietnamese user simulator with a specific persona type.

        Args:
            persona_type: Type of persona to simulate ("detailed", "minimal", "impatient", "confused", or "random")
        """
        self.persona_type = (
            persona_type
            if persona_type != "random"
            else random.choice(["detailed", "minimal", "impatient", "confused"])
        )

        # Initialize persona traits
        self._initialize_persona()

        # Track conversation state
        self.conversation_history = []
        self.provided_contact_info = False
        self.product_interest_stated = False
        self.brand_preference_stated = False
        self.requirements_stated = False
        self.ready_to_purchase = False

    def _initialize_persona(self):
        """Initialize persona details based on selected type"""
        # Common Vietnamese names
        first_names = [
            "Minh",
            "Linh",
            "Huy",
            "Trang",
            "Dung",
            "Phương",
            "Tuấn",
            "Hà",
            "Anh",
            "Thảo",
        ]
        last_names = [
            "Nguyễn",
            "Trần",
            "Lê",
            "Phạm",
            "Hoàng",
            "Huỳnh",
            "Vũ",
            "Võ",
            "Đặng",
            "Bùi",
        ]

        # Base persona details
        self.name = f"{random.choice(last_names)} {random.choice(first_names)}"
        self.email = (
            f"{self.name.lower().replace(' ', '')}{random.randint(1, 999)}@gmail.com"
        )
        self.phone = f"09{random.randint(10000000, 99999999)}"

        # Budget range in Vietnamese Dong (VND)
        budget_ranges = {
            "phone": (2_000_000, 40_000_000),  # 2-40 million VND
            "laptop": (10_000_000, 60_000_000),  # 10-60 million VND
        }

        # Product preferences
        self.product_type = random.choice(["phone", "laptop"])
        self.budget = random.randint(*budget_ranges[self.product_type])

        # Brands based on product type
        brands = {
            "phone": ["Samsung", "iPhone", "Oppo", "Xiaomi", "Vivo", "Realme"],
            "laptop": ["Dell", "HP", "Asus", "Acer", "Lenovo", "MacBook"],
        }
        self.preferred_brand = random.choice(brands[self.product_type])

        # Requirements based on product type
        if self.product_type == "phone":
            self.requirements = {
                "camera": random.randint(12, 108),  # MP
                "storage": random.choice([64, 128, 256, 512, 1024]),  # GB
                "ram": random.choice([4, 6, 8, 12, 16]),  # GB
                "battery": random.randint(3000, 5000),  # mAh
                "screen_size": round(random.uniform(5.5, 7.0), 1),  # inches
            }
        else:  # laptop
            self.requirements = {
                "processor": random.choice(
                    [
                        "Intel i3",
                        "Intel i5",
                        "Intel i7",
                        "Intel i9",
                        "AMD Ryzen 5",
                        "AMD Ryzen 7",
                    ]
                ),
                "storage": random.choice([256, 512, 1024, 2048]),  # GB
                "ram": random.choice([8, 16, 32, 64]),  # GB
                "screen_size": random.choice([13, 14, 15.6, 16, 17]),  # inches
                "graphics": random.choice(
                    ["Integrated", "Nvidia GTX", "Nvidia RTX", "AMD Radeon"]
                ),
            }

        # Persona-specific traits
        if self.persona_type == "detailed":
            self.verbosity = "high"
            self.patience = "high"
            self.technical_knowledge = "high"
            self.willingness_to_share_info = "high"
        elif self.persona_type == "minimal":
            self.verbosity = "low"
            self.patience = "medium"
            self.technical_knowledge = "medium"
            self.willingness_to_share_info = "medium"
        elif self.persona_type == "impatient":
            self.verbosity = "medium"
            self.patience = "low"
            self.technical_knowledge = "medium"
            self.willingness_to_share_info = "low"
        elif self.persona_type == "confused":
            self.verbosity = "high"
            self.patience = "medium"
            self.technical_knowledge = "low"
            self.willingness_to_share_info = "medium"

    def _get_vietnamese_formality(self) -> Dict[str, str]:
        """Get Vietnamese formality based on persona"""
        # Choose pronouns based on persona
        # In Vietnamese, pronouns vary greatly and indicate social relationships
        options = [
            {"self": "Tôi", "you": "bạn"},  # Neutral/formal
            {"self": "Em", "you": "anh/chị"},  # Younger person to older
            {"self": "Mình", "you": "bạn"},  # Friendly/informal
        ]
        return random.choice(options)

    def generate_response(self, agent_message: str) -> str:
        """
        Generate a user response to the workflow agent's message.

        Args:
            agent_message: The message from the workflow agent

        Returns:
            The simulated user's response
        """
        # Add agent message to conversation history
        self.conversation_history.append(
            {"role": "assistant", "content": agent_message}
        )

        # Determine appropriate response based on conversation state and persona
        system_prompt = self._create_system_prompt()

        # Create messages for the API call
        messages = [
            {"role": "system", "content": system_prompt},
            *self.conversation_history,
        ]

        # Get response from OpenAI
        response = client.chat.completions.create(
            model="gpt-4o",  # Or another appropriate model
            messages=messages,
            temperature=0.9 if self.persona_type == "confused" else 0.7,
            max_tokens=150,
        )

        user_response = response.choices[0].message.content

        # Update conversation history with user's response
        self.conversation_history.append({"role": "user", "content": user_response})

        # Update conversation state based on response
        self._update_conversation_state(user_response)

        return user_response

    def _create_system_prompt(self) -> str:
        """Create a system prompt for the API based on persona and conversation state"""
        formality = self._get_vietnamese_formality()

        prompt = f"""You are simulating a Vietnamese user who is interested in purchasing {"a phone" if self.product_type == "phone" else "a laptop"}.

PERSONA DETAILS:
- Type: {self.persona_type} 
- Name: {self.name}
- Product interest: {self.product_type}
- Preferred brand: {self.preferred_brand}
- Budget: {self.budget:,} VND (about {round(self.budget/23000, 2)} USD)
- Technical knowledge: {self.technical_knowledge}
- Patience level: {self.patience}
- Verbosity: {self.verbosity}
- Willingness to share contact info: {self.willingness_to_share_info}
- Vietnamese pronouns: refers to self as '{formality["self"]}', refers to agent as '{formality["you"]}'

Your responses should be primarily in Vietnamese with occasional English words (as is common in Vietnamese tech conversations). 
Use Vietnamese chat style and abbreviations when appropriate:
- "ko" for "không" (no)
- "ok" or "oki" for agreement
- "đc" for "được" (can/ok)
- "khoảng" abbreviated as "khoang" 
- Numbers + "tr" for millions (e.g., "15 tr" = 15 million VND)

PERSONA BEHAVIOR GUIDELINES:
"""

        # Add persona-specific behavior guidelines
        if self.persona_type == "detailed":
            prompt += """
- Provide detailed information about your needs and preferences
- Ask technical questions about the products
- Respond directly to questions and provide thorough information when asked
- Be polite and patient
- Willing to share contact information when it makes sense
"""
        elif self.persona_type == "minimal":
            prompt += """
- Keep responses very short, often just a few words
- Answer questions directly with minimal elaboration
- Sometimes use single word answers or short Vietnamese phrases
- Prefer simple language and avoid technical details
- Hesitant to share contact information immediately but will if pressed
"""
        elif self.persona_type == "impatient":
            prompt += """
- Show signs of impatience with phrases like "nhanh lên", "mau", etc.
- Ask about prices early and often
- Focus heavily on getting the best deal
- May sound abrupt or slightly demanding
- Reluctant to share contact information until convinced of good value
- Might try to negotiate or ask about discounts
"""
        elif self.persona_type == "confused":
            prompt += """
- Ask for clarification frequently
- Sometimes misunderstand basic product specifications
- Change your mind about requirements mid-conversation
- Ask repeated questions about the same topics
- Get details wrong or confused (like mixing up RAM and storage)
- Somewhat hesitant to share contact information due to confusion
"""

        # Add information about product requirements
        prompt += f"\nPRODUCT REQUIREMENTS:\n"
        for key, value in self.requirements.items():
            formatted_key = key.replace("_", " ").title()
            if key == "camera":
                prompt += f"- {formatted_key}: {value}MP hoặc cao hơn\n"
            elif key in ["storage", "ram"]:
                prompt += f"- {formatted_key}: {value}GB hoặc cao hơn\n"
            elif key == "battery":
                prompt += f"- {formatted_key}: {value}mAh hoặc cao hơn\n"
            elif key == "screen_size":
                prompt += f"- {formatted_key}: {value} inch hoặc lớn hơn\n"
            else:
                prompt += f"- {formatted_key}: {value}\n"

        # Conversation state-specific guidance
        prompt += "\nCONVERSATION STATE GUIDELINES:\n"

        if not self.product_interest_stated:
            prompt += (
                "- When asked about product type, indicate your interest in "
                + self.product_type
                + "\n"
            )

        if self.product_interest_stated and not self.brand_preference_stated:
            prompt += f"- If asked about brand preference, mention {self.preferred_brand} or say you're open to suggestions\n"

        if not self.requirements_stated:
            prompt += "- Be ready to mention your requirements when the conversation moves to specific product features\n"

        if not self.provided_contact_info:
            if self.willingness_to_share_info == "high":
                prompt += "- Provide your contact info (email and phone) when asked or when it seems appropriate\n"
            elif self.willingness_to_share_info == "medium":
                prompt += "- Initially hesitate but eventually provide contact info when asked repeatedly\n"
            else:
                prompt += "- Be very reluctant to share contact info; may need significant convincing\n"

        prompt += """
IMPORTANT:
- Respond ONLY as the user would, do not include explanations or meta-commentary
- Your response should look like a genuine Vietnamese customer chat message
- Do not use quotation marks around your response
- Sometimes include typos as would be natural in chat
- If the agent asks for your contact information and you're ready to provide it, give your name, email, and phone number
"""

        return prompt

    def _update_conversation_state(self, response: str) -> None:
        """Update conversation state based on the user's response"""
        response_lower = response.lower()

        # Check if product interest is stated
        if not self.product_interest_stated:
            if (
                "điện thoại" in response_lower
                or "phone" in response_lower
                or "smartphone" in response_lower
            ):
                self.product_interest_stated = True
                self.product_type = "phone"
            elif (
                "laptop" in response_lower
                or "máy tính" in response_lower
                or "notebook" in response_lower
            ):
                self.product_interest_stated = True
                self.product_type = "laptop"

        # Check if brand preference is stated
        if self.product_interest_stated and not self.brand_preference_stated:
            brands = [
                "samsung",
                "iphone",
                "apple",
                "oppo",
                "xiaomi",
                "vivo",
                "realme",
                "dell",
                "hp",
                "asus",
                "acer",
                "lenovo",
                "macbook",
            ]
            for brand in brands:
                if brand in response_lower:
                    self.brand_preference_stated = True
                    self.preferred_brand = brand
                    break

        # Check if contact info is provided
        if not self.provided_contact_info:
            # Check for email patterns
            if "@" in response_lower and (
                ".com" in response_lower or ".vn" in response_lower
            ):
                self.provided_contact_info = True

            # Check for phone number patterns
            phone_patterns = ["09", "08", "07", "03", "05", "+84"]
            for pattern in phone_patterns:
                if pattern in response_lower and any(
                    c.isdigit() for c in response_lower
                ):
                    self.provided_contact_info = True
                    break

        # Check if requirements are stated
        if not self.requirements_stated:
            requirement_keywords = [
                "ram",
                "bộ nhớ",
                "camera",
                "pin",
                "màn hình",
                "screen",
                "processor",
                "cpu",
                "storage",
                "ổ cứng",
                "ssd",
                "hdd",
            ]
            for keyword in requirement_keywords:
                if keyword in response_lower:
                    self.requirements_stated = True
                    break


def simulate_conversation(workflow_agent, user_simulator, max_turns=10):
    """
    Simulate a conversation between the workflow agent and user simulator

    Args:
        workflow_agent: Function that takes a user message and returns an agent response
        user_simulator: VietnameseUserSimulator instance
        max_turns: Maximum number of conversation turns

    Returns:
        List of conversation turns
    """
    conversation = []

    # Start conversation with a greeting from the workflow agent
    agent_message = "Xin chào! Tôi có thể giúp gì cho bạn hôm nay?"
    conversation.append({"role": "assistant", "content": agent_message})

    # Initial user message
    initial_messages = [
        "Tôi muốn mua điện thoại mới",
        "Cần tư vấn laptop",
        "Em đang tìm điện thoại",
        "Chào, cho hỏi laptop giá tốt",
        "Mình đang cần một chiếc điện thoại mới",
    ]
    user_message = user_simulator.generate_response(agent_message)
    conversation.append({"role": "user", "content": user_message})

    # Continue conversation for up to max_turns
    for _ in range(max_turns - 1):
        # Get agent response
        agent_message = workflow_agent(user_message)
        conversation.append({"role": "assistant", "content": agent_message})

        # Check if conversation should end
        if (
            "liên hệ" in agent_message.lower()
            and "cảm ơn" in agent_message.lower()
            and user_simulator.provided_contact_info
        ):
            # Agent has thanked user for contact info and promised follow-up
            break

        # Get user response
        user_message = user_simulator.generate_response(agent_message)
        conversation.append({"role": "user", "content": user_message})

    return conversation


# Example usage:
def example_workflow_agent(user_message):
    """
    Example workflow agent implementation (placeholder for your actual agent)
    This is just to demonstrate how to use the user simulator
    """
    # This is where you would connect to your actual workflow agent
    # For demonstration purposes, we'll use a simple rule-based response

    if "mua" in user_message.lower() or "tìm" in user_message.lower():
        return "Bạn quan tâm đến điện thoại hay laptop?"

    if "điện thoại" in user_message.lower() or "phone" in user_message.lower():
        return (
            "Bạn có quan tâm đến thương hiệu cụ thể nào không? Samsung, iPhone, Oppo?"
        )

    if "laptop" in user_message.lower():
        return (
            "Bạn có quan tâm đến thương hiệu cụ thể nào không? Dell, HP, Asus, Lenovo?"
        )

    if any(
        brand in user_message.lower()
        for brand in ["samsung", "iphone", "oppo", "dell", "hp", "asus"]
    ):
        return "Bạn có thể cho biết thêm về nhu cầu sử dụng và ngân sách của mình không? Và cho mình xin thông tin liên hệ của bạn được không?"

    if "@" in user_message.lower() or "09" in user_message or "08" in user_message:
        return "Cảm ơn bạn đã cung cấp thông tin. Đội ngũ tư vấn của chúng tôi sẽ liên hệ với bạn trong thời gian sớm nhất."

    return "Bạn có thể cho mình biết thêm thông tin về nhu cầu của bạn và để lại thông tin liên hệ được không?"


# Run simulation
if __name__ == "__main__":
    # Create different persona types for demonstration
    personas = ["detailed", "minimal", "impatient", "confused"]

    for persona in personas:
        print(f"\n{'='*50}")
        print(f"SIMULATING CONVERSATION WITH {persona.upper()} PERSONA")
        print(f"{'='*50}\n")

        user_sim = VietnameseUserSimulator(persona_type=persona)
        conversation = simulate_conversation(example_workflow_agent, user_sim)

        print(f"Persona: {persona}")
        print(f"Name: {user_sim.name}")
        print(f"Product: {user_sim.product_type}")
        print(f"Brand: {user_sim.preferred_brand}")
        print(f"Budget: {user_sim.budget:,} VND")
        print("\nCONVERSATION:")

        for turn in conversation:
            if turn["role"] == "assistant":
                print(f"\nAgent: {turn['content']}")
            else:
                print(f"\nUser: {turn['content']}")

        print(f"\nContact info provided: {user_sim.provided_contact_info}")
        print(f"Product interest stated: {user_sim.product_interest_stated}")
        print(f"Brand preference stated: {user_sim.brand_preference_stated}")
        print(f"Requirements stated: {user_sim.requirements_stated}")

2025-05-13 22:02:21,668 - __main__ - INFO - Starting Mobile Phone Store Chatbot Evaluation
2025-05-13 22:02:21,670 - __main__ - INFO - Starting simulation 1/10
2025-05-13 22:02:21,673 - __main__ - INFO - Generated user info: {"name": "Hoàng Trang", "gender": "Nam", "phone": "0335526884", "email": "trang.hoang1608@yahoo.com", "scenario": "Đang tìm một chiếc điện thoại mới với camera tốt"}
2025-05-13 22:02:29,817 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-13 22:02:29,832 - __main__ - INFO - Created user persona: {"name": "Hoàng Trang", "gender": "Nam", "phone": "0335526884", "email": "trang.hoang1608@yahoo.com", "scenario": "Đang tìm một chiếc điện thoại mới với camera tốt", "userPersona": {"name": "Hoàng Trang", "gender": "Nam", "phoneNumber": "0335526884", "email": "trang.hoang1608@yahoo.com", "age": 28, "occupation": "Nhân viên marketing", "preferredPhoneBrands": ["Samsung", "Apple", "Xiaomi"], "previousPhones": ["Samsung G

KeyboardInterrupt: 